In [0]:
%pip install -U -qqqq mlflow databricks-openai databricks-agents
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("catalog_name", "btafur_catalog")
dbutils.widgets.text("schema_name", "default")
dbutils.widgets.text("model_name", "quickstart_agent")
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
model_name = dbutils.widgets.get("model_name")

dbutils.widgets.text("experiment_name", f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/{model_name}_{catalog_name}")
experiment_name = dbutils.widgets.get("experiment_name")

In [0]:
import mlflow
import mlflow.genai.datasets
import time
from databricks.connect import DatabricksSession

mlflow.set_experiment(experiment_name)

In [0]:
query = f"SELECT * FROM {catalog_name}.{schema_name}.sentences"
df = spark.sql(query).toPandas()

In [0]:

model_uri = f"models:/{catalog_name}.{schema_name}.{model_name}@champion"
agent = mlflow.pyfunc.load_model(model_uri)

def predict_sentiment(review):
    messages = [{
        "role": "user",
        "content": f"{review}"
    }]
    return agent.predict({"messages": messages})

df['predictions'] = df['review'].apply(predict_sentiment)
display(df)
